In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [8]:
bike = pd.read_csv("Bike Sharing Dataset.csv", usecols=[2,5,8,9,10,16])
bike.info()
bike.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17379 non-null  object 
 1   hr          17379 non-null  int64  
 2   workingday  17379 non-null  object 
 3   weathersit  17379 non-null  object 
 4   temp        17379 non-null  float64
 5   cnt         17379 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 814.8+ KB


,season,hr,workingday,weathersit,temp,cnt
0,spring,0,no,clear,0.24,16
1,spring,1,no,clear,0.22,40
2,spring,2,no,clear,0.22,32
3,spring,3,no,clear,0.24,13
4,spring,4,no,clear,0.24,1


In [9]:
bike.info()
print(bike.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17379 non-null  object 
 1   hr          17379 non-null  int64  
 2   workingday  17379 non-null  object 
 3   weathersit  17379 non-null  object 
 4   temp        17379 non-null  float64
 5   cnt         17379 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 814.8+ KB
season        0
hr            0
workingday    0
weathersit    0
temp          0
cnt           0
dtype: int64


In [12]:
bike['workingday2'] = bike.workingday.factorize()[0]
bike['weathersit2'] = bike.weathersit.factorize()[0]
est = smf.ols('cnt ~ hr + workingday2 + temp', bike).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     2259.
Date:                Fri, 03 Nov 2023   Prob (F-statistic):               0.00
Time:                        10:37:58   Log-Likelihood:            -1.1218e+05
No. Observations:               17379   AIC:                         2.244e+05
Df Residuals:                   17375   BIC:                         2.244e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -84.6416      3.906    -21.669      0.000     -92.298     -76.985
hr              9.0494      0.170     53.100      0.000       8.715       9.383
workingday2     3.7938      2.512      1.511      0.131      -1.129       8.717
temp          336.0729      6.129     54.834      0.000     324.060     348.086
==============================================================================
Omnibus:                     3951.373   Durbin-Watson:                   0.484
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8470.361
Skew:                           1.324   Prob(JB):                         0.00
Kurtosis:                       5.164   Cond. No.                         78.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""